In [7]:
import pickle as pk
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from glob import glob
%pylab inline
import datajoint as dj
from minio import Minio
import json
import yaml
import sys, os

sys.path.append('./lib')
from utilities import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [4]:
structureTable = Structure.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,id,structure,positive_s3_fp,negative_s3_fp,positive_img_fp,negative_img_fp
0,0,5N,CSHL_cells_features/MD589/Properties/5N/MD589_...,CSHL_cells_features/MD589/Properties/5N/MD589_...,CSHL_cells_features/MD589/Images/5N/MD589_5N_p...,CSHL_cells_features/MD589/Images/5N/MD589_5N_n...
1,1,6N,CSHL_cells_features/MD589/Properties/6N/MD589_...,CSHL_cells_features/MD589/Properties/6N/MD589_...,CSHL_cells_features/MD589/Images/6N/MD589_6N_p...,CSHL_cells_features/MD589/Images/6N/MD589_6N_n...
2,2,7N,CSHL_cells_features/MD589/Properties/7N/MD589_...,CSHL_cells_features/MD589/Properties/7N/MD589_...,CSHL_cells_features/MD589/Images/7N/MD589_7N_p...,CSHL_cells_features/MD589/Images/7N/MD589_7N_n...
3,3,7n,CSHL_cells_features/MD589/Properties/7n/MD589_...,CSHL_cells_features/MD589/Properties/7n/MD589_...,CSHL_cells_features/MD589/Images/7n/MD589_7n_p...,CSHL_cells_features/MD589/Images/7n/MD589_7n_n...
4,4,Amb,CSHL_cells_features/MD589/Properties/Amb/MD589...,CSHL_cells_features/MD589/Properties/Amb/MD589...,CSHL_cells_features/MD589/Images/Amb/MD589_Amb...,CSHL_cells_features/MD589/Images/Amb/MD589_Amb...
5,5,LC,CSHL_cells_features/MD589/Properties/LC/MD589_...,CSHL_cells_features/MD589/Properties/LC/MD589_...,CSHL_cells_features/MD589/Images/LC/MD589_LC_p...,CSHL_cells_features/MD589/Images/LC/MD589_LC_n...
6,6,LRt,CSHL_cells_features/MD589/Properties/LRt/MD589...,CSHL_cells_features/MD589/Properties/LRt/MD589...,CSHL_cells_features/MD589/Images/LRt/MD589_LRt...,CSHL_cells_features/MD589/Images/LRt/MD589_LRt...
7,7,Pn,CSHL_cells_features/MD589/Properties/Pn/MD589_...,CSHL_cells_features/MD589/Properties/Pn/MD589_...,CSHL_cells_features/MD589/Images/Pn/MD589_Pn_p...,CSHL_cells_features/MD589/Images/Pn/MD589_Pn_n...
8,8,Tz,CSHL_cells_features/MD589/Properties/Tz/MD589_...,CSHL_cells_features/MD589/Properties/Tz/MD589_...,CSHL_cells_features/MD589/Images/Tz/MD589_Tz_p...,CSHL_cells_features/MD589/Images/Tz/MD589_Tz_n...
9,9,VLL,CSHL_cells_features/MD589/Properties/VLL/MD589...,CSHL_cells_features/MD589/Properties/VLL/MD589...,CSHL_cells_features/MD589/Images/VLL/MD589_VLL...,CSHL_cells_features/MD589/Images/VLL/MD589_VLL...


In [9]:
#Parameters
param = {}
param['max_depth']= 3   # depth of tree
param['eta'] = 0.2      # shrinkage parameter
param['silent'] = 1     # not silent
param['objective'] = 'binary:logistic' #'multi:softmax'
param['nthread'] = 7 # Number of threads used
param['num_class']=1

In [10]:
def get_scores(X_train, X_valid, y_train, param):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_valid)
    num_round = 100
    bst = xgb.train(param, dtrain, num_round, verbose_eval=False)
    score = bst.predict(dval, output_margin=True, ntree_limit=bst.best_ntree_limit)
    score = np.around(score, 3)
    
    return score

In [65]:
structure = 'SNC'
posi_fp = os.environ['ROOT_DIR'] + strucDF[strucDF['structure']==structure]['positive_s3_fp'].to_list()[0]
nega_fp = os.environ['ROOT_DIR'] + strucDF[strucDF['structure']==structure]['negative_s3_fp'].to_list()[0]
posi = pd.read_pickle(posi_fp)
nega = pd.read_pickle(nega_fp)
len(posi),len(nega)

(13474, 102548)

In [66]:
features = []
labels = []
fp = []
fp.append(nega_fp)
fp.append(posi_fp)
for state in range(2):
    cells = pd.read_pickle(fp[state])
    cells = cells.drop(['left','top'],1)
    cells = np.asarray(cells)
    for i in range(len(cells)):
        cells[i][0] = cells[i][0][:10]
    features.extend(np.concatenate((np.array(list(cells[:,0])),cells[:,1:]),axis=1))
    labels.extend([state]*len(cells))
#print(time()-t0)
features = np.array(features)
labels = np.array(labels)
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.30, random_state=6)
y_pred_valid = get_scores(X_train, X_valid, y_train, param)
pred = y_pred_valid>0
#pred = get_predictions(y_pred_valid)
acc = sum(pred ==y_valid)/len(y_valid)
print(structure, acc)

SNC 0.8838452035510098


In [64]:
! python FeaturesShow.py MD589 SNC

ALREADY DOWNLOADED FILE
./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
Connecting kui@ucsd-demo-db.datajoint.io:3306
